In [2]:
import dask.dataframe as dd
df = dd.read_csv('../../input/dask-nyc-flights/*.csv',
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})
df.head()

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1990-01-01,1,1621.0,1540,1747.0,1701,US,33,NaN,86.0,...,NaN,46.0,41.0,EWR,PIT,319.0,NaN,NaN,False,0
1,1990-01-02,2,1547.0,1540,1700.0,1701,US,33,NaN,73.0,...,NaN,-1.0,7.0,EWR,PIT,319.0,NaN,NaN,False,0
2,1990-01-03,3,1546.0,1540,1710.0,1701,US,33,NaN,84.0,...,NaN,9.0,6.0,EWR,PIT,319.0,NaN,NaN,False,0
3,1990-01-04,4,1542.0,1540,1710.0,1701,US,33,NaN,88.0,...,NaN,9.0,2.0,EWR,PIT,319.0,NaN,NaN,False,0
4,1990-01-05,5,1549.0,1540,1706.0,1701,US,33,NaN,77.0,...,NaN,5.0,9.0,EWR,PIT,319.0,NaN,NaN,False,0


In [3]:
df

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=10,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float64,int64,float64,int64,object,int64,object,float64,float64,float64,float64,float64,object,object,float64,float64,float64,bool,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
temp = df.groupby(['Date'])['UniqueCarrier','FlightNum']

AttributeError: 'SeriesGroupBy' object has no attribute 'compute'

In [3]:
### Find all tail numbers that have flown multiple flights
routes = df.groupby('TailNum').FlightNum.nunique()

In [4]:
routes.compute()

TailNum
EI-BWD    122
EI-CAL     10
EI-CAM     14
EI-CIW     84
N050AA     35
         ... 
N976TW     13
N978TW      8
N979TW     11
N980TW      6
N982TW      1
Name: FlightNum, Length: 3712, dtype: int64

In [12]:
import dask
%time routes.compute(scheduler='threads', num_workers=1)

CPU times: user 27 s, sys: 2.33 s, total: 29.4 s
Wall time: 28.4 s


TailNum
EI-BWD    122
EI-CAL     10
EI-CAM     14
EI-CIW     84
N050AA     35
         ... 
N976TW     13
N978TW      8
N979TW     11
N980TW      6
N982TW      1
Name: FlightNum, Length: 3712, dtype: int64

In [13]:
%time routes.compute(scheduler='threads', num_workers=2)

CPU times: user 27.8 s, sys: 6.09 s, total: 33.8 s
Wall time: 28.8 s


TailNum
EI-BWD    122
EI-CAL     10
EI-CAM     14
EI-CIW     84
N050AA     35
         ... 
N976TW     13
N978TW      8
N979TW     11
N980TW      6
N982TW      1
Name: FlightNum, Length: 3712, dtype: int64

In [14]:
%time routes.compute(scheduler='threads', num_workers=4)

CPU times: user 29.2 s, sys: 7.92 s, total: 37.1 s
Wall time: 28.8 s


TailNum
EI-BWD    122
EI-CAL     10
EI-CAM     14
EI-CIW     84
N050AA     35
         ... 
N976TW     13
N978TW      8
N979TW     11
N980TW      6
N982TW      1
Name: FlightNum, Length: 3712, dtype: int64

In [9]:
%time routes.compute(num_workers=8)

CPU times: user 35.7 s, sys: 18.5 s, total: 54.1 s
Wall time: 44.2 s


TailNum
EI-BWD    122
EI-CAL     10
EI-CAM     14
EI-CIW     84
N050AA     35
         ... 
N976TW     13
N978TW      8
N979TW     11
N980TW      6
N982TW      1
Name: FlightNum, Length: 3712, dtype: int64